pip install spotify

In [149]:
import time
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id='5c7c9ce33ce246aab5247970b81439f6', client_secret='541c83e188aa4b2aa2a63592f34a29db')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
df = pd.read_csv('spotify_with_genre.csv')

def get_artist_genres(artist_name, retries=3, delay=1):
    for i in range(retries):
        try:
            result = sp.search(q='artist:' + artist_name, type='artist', limit=1)
            items = result['artists']['items']
            if items:
                return items[0]['genres']
            else:
                return []
        except Exception as e:
            print(f"Retry {i+1}/{retries} - Error with artist '{artist_name}': {e}")
            time.sleep(delay)
    return []

# 모든 아티스트에 대해 장르 가져오기
all_track_genres = []
count = 0
for artists_str in df['artist_name']:
    count+=1
    if count == 5:
        break
    artist_names = [artist.strip() for artist in artists_str.split(',')]
    track_genres = set()

    for artist_name in artist_names:
        genres = get_artist_genres(artist_name)
        track_genres.update(genres)
        time.sleep(0.1)  # 요청 사이 딜레이

    if track_genres:
        all_track_genres.append(', '.join(sorted(track_genres)))
    else:
        all_track_genres.append('unknown')

# 결과 저장
print(all_track_genres)
# df['track_genre'] = all_track_genres
print("✅ 장르 추가 완료! → 'spotify_with_all_genres.csv' 로 저장됨")

In [150]:
def get_genres_from_track(track_id, retries=3, delay=1):
    for attempt in range(retries):
        try:
            track = sp.track(track_id)
            artist_ids = [artist['id'] for artist in track['artists']]
            all_genres = set()
            for artist_id in artist_ids:
                artist = sp.artist(artist_id)
                all_genres.update(artist.get('genres', []))
                time.sleep(delay)  # rate limit 보호
            return ', '.join(sorted(all_genres)) if all_genres else 'unknown'
        except Exception as e:
            print(f"[{track_id}] Error (attempt {attempt + 1}): {e}")
            time.sleep(delay)
    return 'unknown'
# # 장르 가져오기 실행
# genres = []
# for track_id in df['track_id']:
#     genre = get_genres_from_track(track_id)
#     genres.append(genre)
#     print(f"✅ {track_id} → {genre}")

In [153]:
for i, row in df[df['track_genre'].isna()].iterrows():
    genre = get_genres_from_track(row['track_id'], 3)
    df.at[i, 'track_genre'] = genre
    print(f"🔄 재시도: {row['track_id']} → {genre}")

In [152]:
df.to_csv('spotify_with_genre.csv', index=False)